In [60]:
# install the requirements
!pip install atproto
!pip install pandas

  Using cached pandas-2.3.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached numpy-2.3.0-cp313-cp313-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.3.0-cp313-cp313-macosx_11_0_arm64.whl (10.7 MB)
Using cached numpy-2.3.0-cp313-cp313-macosx_14_0_arm64.whl (5.1 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]


In [61]:
# load requirements
from atproto import Client
import pandas as pd

In [2]:
# create a client instance
client = Client()

# get the app password
with open("app_password.txt", "r") as f:
    app_password = f.read()

handle = "mxwlnd.bsky.social"

# login with my credentials
client.login(handle, app_password)

ProfileViewDetailed(did='did:plc:5sqqg66p7muc7ogbp6xx4sw6', handle='mxwlnd.bsky.social', associated=ProfileAssociated(chat=None, feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:5sqqg66p7muc7ogbp6xx4sw6/bafkreigwrjedzb7jvmowkn6fbe2atbnlwecsa4ouk5wpz54eg6rqkvayrq@jpeg', banner=None, created_at='2025-05-19T19:28:35.738Z', description=None, display_name='', followers_count=2, follows_count=1, indexed_at='2025-05-19T19:28:35.738Z', joined_via_starter_pack=None, labels=[], pinned_post=None, posts_count=0, verification=None, viewer=ViewerState(blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=None, muted=False, muted_by_list=None, py_type='app.bsky.actor.defs#viewerState'), py_type='app.bsky.actor.defs#profileViewDetailed')

In [92]:
# define a function that returns a df with all posts from given handle

def retrieve_posts(handle):
    did = client.com.atproto.identity.resolve_handle({'handle': handle})['did']
    cursor = None
    all_posts = []
    all_dates = []
    keep_looping = True

    while keep_looping == True:
        response = client.app.bsky.feed.get_author_feed({
            'actor': did,
            'cursor': cursor,
            'limit': 100
        })

        # get the response feed
        feed = response['feed']

        for item in feed:
            post_handle = item["post"]["author"]["handle"]
            text = item["post"]["record"]["text"]
            date = item["post"]["record"]["created_at"]

            # filter out reposts
            if handle == post_handle:
                all_posts.append(text)
                all_dates.append(date)

        cursor = response['cursor']
        if not cursor:
            keep_looping = False

    df = pd.DataFrame({"text": all_posts, "date": all_dates}).sort_values(by="date", ascending=False)
    

    return df

df = retrieve_posts('britta-hasselmann.de')
